In [1]:
# install packages
!pip install -q keras-nlp tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.1/792.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 73.5 MB/s eta 0:00:00


In [2]:
# import libraries
import tensorflow as tf
import keras_nlp
from sklearn.model_selection import train_test_split

# sample dataset: product reviews with binary labels (1=positive, 0=negative)
review_samples = [
    "Highly recommended product.",
    "Worst purchase ever.",
    "Very useful and reliable.",
    "Broke within a week.",
    "Exceeded my expectations.",
    "Won't buy again.",
    "Good value for the price.",
    "Total disappointment."
]
sentiment_labels = [1, 0, 1, 0, 1, 0, 1, 0]

In [3]:
# split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    review_samples, sentiment_labels, test_size=0.25, random_state=42
)

# load tokenizer and pretrained model backbone
text_tokenizer = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en_uncased")
bert_backbone = keras_nlp.models.BertBackbone.from_preset("bert_base_en_uncased")

In [4]:
# build classifier model using the backbone and tokenizer
text_classifier = keras_nlp.models.BertClassifier(
    backbone=bert_backbone,
    num_classes=2,
    preprocessor=text_tokenizer
)

In [5]:
# compile the model with optimizer and loss function
text_classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [6]:
# prepare data pipelines
train_data = tf.data.Dataset.from_tensor_slices((train_texts, train_labels)).batch(2)
val_data = tf.data.Dataset.from_tensor_slices((val_texts, val_labels)).batch(2)

# fine-tune the classifier
text_classifier.fit(train_data, validation_data=val_data, epochs=3)

Epoch 1/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 44s 4s/step - accuracy: 0.6458 - loss: 0.6336 - val_accuracy: 0.0000e+00 - val_loss: 0.8973
Epoch 2/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - accuracy: 1.0000 - loss: 0.5497 - val_accuracy: 0.0000e+00 - val_loss: 0.9561
Epoch 3/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - accuracy: 1.0000 - loss: 0.4321 - val_accuracy: 0.0000e+00 - val_loss: 0.9179
